In [6]:
# pip install -q tqdm

Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import glob
import torch
import random
import traceback


import numpy as np



from PIL import Image
from tqdm.notebook import tqdm
from torchvision import transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [2]:
from src import ROOT_DIR

Data Class

In [ ]:
class FallVideoDataset(Dataset):
    def __init__(self, root_dir, transform=None, max_frames=200):
        """
        Args:
            root_dir (str): Root directory of the dataset.
            transform (callable, optional): Transform to be applied on a frame.
            max_frames (int): Number of frames per video to use (for padding/trimming).
        """
        self.root_dir = root_dir
        self.transform = transform
        self.samples = []
        self.max_frames = max_frames


        # # Label: 1 for falling, 0 for not falling
        for subject in os.listdir(root_dir):
            subject_folder = os.path.join(root_dir, subject)
            
            for each_action in os.listdir(subject_folder):
                action_folder = os.path.join(subject_folder, each_action)
                label = 1 if 'fall' in each_action.lower() else 0
                
                video_folder = action_folder # as each action folder is made up of frames from a video
                
                if os.path.isdir(video_folder):
                    self.samples.append((video_folder, label))
                
                     

    def __len__(self):
        return len(self.samples)
    
    def detailed_sample_stats(self):
        all_files = [os.listdir(video_folder) for video_folder, _ in self.samples]
        all_files = [item for sublist in all_files for item in sublist]
        frames_only = [file for file in all_files if file.endswith('g')]
        
        temp = {}
        
        for video_folder, _ in self.samples:
            temp[str(os.path.basename(video_folder))] = len(os.listdir(video_folder))
        
        return {
            'videos': len(self.samples), 
            'frames': len(frames_only),
            'frames_per_video': temp
        }

    def load_frames(self, video_folder):
        frames = sorted(os.listdir(video_folder))  
        frame_groups = {}

        # group by prefix before "_aug"
        for f in frames:
            if f.lower().endswith((".jpg", ".jpeg", ".png")):
                base = f.split("_aug")[0]  # e.g. "frame_001"
                frame_groups.setdefault(base, []).append(f)

        frame_tensors = []
        selected_keys = sorted(frame_groups.keys())[:self.max_frames]

        for key in selected_keys:
            variants = frame_groups[key]
            frame_choice = random.choice(variants)  # pick one randomly
            frame_path = os.path.join(video_folder, frame_choice)

            image = Image.open(frame_path).convert("RGB")
            if self.transform:
                image = self.transform(image)
            frame_tensors.append(image)

        # padding
        while len(frame_tensors) < self.max_frames:
            frame_tensors.append(torch.zeros_like(frame_tensors[0]))

        return torch.stack(frame_tensors)  # [T, C, H, W]

    def __getitem__(self, idx):
        video_path, label = self.samples[idx]
        video_tensor = self.load_frames(video_path)
        return video_tensor, torch.tensor(label, dtype=torch.long)


In [4]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

dataset = FallVideoDataset(root_dir="D:\Fall Dataset\Dataset CAUCAFall_1\Dataset CAUCAFall\CAUCAFall", transform=transform)



In [5]:
dataset.detailed_sample_stats()

{'videos': 100,
 'frames': 20001,
 'frames_per_video': {'Fall backwards': 508,
  'Fall forward': 300,
  'Fall left': 418,
  'Fall right': 382,
  'Fall sitting': 436,
  'Hop': 428,
  'Kneel': 436,
  'Pick up object': 444,
  'Sit down': 480,
  'Walk': 472}}

In [7]:

loader = DataLoader(dataset, batch_size=4, shuffle=True)

for batch in loader:
    print(type(batch))
    video, label = batch
    print(video.shape)
    print(label)
    break

<class 'list'>
torch.Size([4, 20, 3, 224, 224])
tensor([0, 1, 0, 0])


Model 

In [8]:
import torch.nn as nn
import torchvision.models as models

In [9]:
model = models.resnet18(pretrained=True)

c:\Users\Dell\anaconda3\envs\image_realignment\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Dell\anaconda3\envs\image_realignment\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [10]:
import torch
import torch.nn as nn

class SimpleAttention(nn.Module):
    def __init__(self, embed_dim):
        super(SimpleAttention, self).__init__()
        self.query = nn.Linear(embed_dim, embed_dim)
        self.key   = nn.Linear(embed_dim, embed_dim)
        self.value = nn.Linear(embed_dim, embed_dim)
        
    def forward(self, x):
        # x shape: [batch, seq_len, embed_dim]
        Q = self.query(x)
        K = self.key(x)
        V = self.value(x)

        # Compute attention scores
        scores = torch.matmul(Q, K.transpose(-2, -1)) / (K.size(-1) ** 0.5)
        attn_weights = torch.softmax(scores, dim=-1)

        # Apply attention weights
        output = torch.matmul(attn_weights, V)
        return output


In [11]:
class VideoClassificationModel(nn.Module):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.model = models.resnet18(pretrained=True)
        self.model = self.config(self.model)
        
        self.rnn = self.define_rnn_cell()
        
        self.attention = SimpleAttention(kwargs['hidden_size'])
        
        self.fc = self.fc_layers(kwargs['hidden_size'])
        
        
    def config(self, model):
        # Remove the classification head (fc layer)
        modules = list(model.children())[:-1]  # remove last layer
        feature_extractor = nn.Sequential(*modules)
        for param in feature_extractor.parameters():
            param.requires_grad = False
        
        return feature_extractor
    
    
    def define_rnn_cell(self, input_size=512, hidden_size=256, cell_type="LSTM"):
        """Defines an RNN cell (default: LSTM)."""
        if cell_type == "LSTM":
            return nn.LSTM(input_size=input_size, hidden_size=hidden_size)  
        elif cell_type == "GRU":
            return nn.GRU(input_size=input_size, hidden_size=hidden_size)  
        else:
            return nn.RNN(input_size=input_size, hidden_size=hidden_size)
        
        
    def fc_layers(self, input_size, num_classes):
        fc_layer = nn.Sequential(
            nn.Linear(input_size, input_size // 2),
            nn.ReLU(),
            nn.Linear(input_size // 2, num_classes)
        )
        
        return fc_layer
    
    def forward(self, x):
        
        batch_size, seq_len, C, H, W = x.size()
    
        # (1) Extract CNN features for each frame
        x = x.view(batch_size * seq_len, C, H, W)          # merge batch & time
        features = self.model(x)                           # (batch*seq, 512, 1, 1)
        features = features.view(batch_size, seq_len, -1)  # (batch, seq, 512)
        
        # (2) RNN over sequence
        rnn_out, _ = self.rnn(features)                    # (batch, seq, hidden)
        
        # (3) Apply attention over sequence
        attn_out = self.attention(rnn_out)                 # (batch, seq, hidden)
        attn_out = attn_out.mean(dim=1)                    # reduce sequence → (batch, hidden)
        
        # (4) Final classification
        output = self.fc(attn_out)                         # (batch, num_classes)
        return output
        
        

In [6]:
pip install albumentations

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 369.4/369.4 KB 6.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 53.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 67.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 646.2/646.2 KB 84.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.3
    Uninstalling numpy-1.24.3:
      Successfully uninstalled numpy-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
norfair 2.3.0 requires numpy<2.0.0,>=1.23.0; python_version >= "3.8", but you have numpy 2.2.6 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [1]:
import os
import cv2
import albumentations as A
from tqdm import tqdm

# Define heavy augmentations (to save offline)
augment = A.Compose([
    A.RandomBrightnessContrast(p=0.5),
    A.HueSaturationValue(p=0.5),
    A.MotionBlur(blur_limit=7, p=0.3),
    A.GaussianBlur(blur_limit=5, p=0.3),
    A.RandomGamma(p=0.5),
    A.CLAHE(p=0.3),  # adaptive histogram equalization
    A.Perspective(scale=(0.05,0.1), p=0.3),
])



In [2]:
def augmentation_naming_convention(frame, aug_count):
    if frame.endswith((".jpg", ".jpeg", ".png")):
        frame_name = frame.split('.')[0]
    
    else:
        frame_name = frame
        
    final_frame_name = frame_name + '_aug_' + str(aug_count)
    
    return final_frame_name
    

def augment_video_frames(input_folder, output_folder, num_aug=3):
    """
    input_folder: original video frames (e.g., person1_fall)
    output_folder: where augmented versions will be stored
    num_aug: how many augmented versions to create
    """
    frames = sorted(glob.glob(f"{input_folder}/*.*g"))
    # os.makedirs(output_folder, exist_ok=True)
    
    # augmented_frames = sorted(glob.glob(f"{input_folder}/*_aug_*.*g"))
    
    # if len(augmented_frames) :
    #     return 

    for frame_name in tqdm(frames):
        frame_path = os.path.join(input_folder, frame_name)
        image = cv2.imread(frame_path)
        if image is None:
            raise FileExistsError(f"File path doesn't exists : {frame_path}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        
        for n in range(1, num_aug+1):  # multiple augmented versions
            # Apply augmentation
            augmented = augment(image=image)
            aug_img = augmented["image"]
            temp_frame_name, ext = os.path.splitext(frame_name)
            updated_frame_name = augmentation_naming_convention(temp_frame_name, n)
            updated_frame_name += ext
            # aug_subfolder = os.path.join(output_folder, f"aug{n+1}")
            # os.makedirs(aug_subfolder, exist_ok=True)

            # Save
            save_path = os.path.join(output_folder, updated_frame_name)
            if os.path.exists(save_path):
                return
            cv2.imwrite(save_path, cv2.cvtColor(aug_img, cv2.COLOR_RGB2BGR))


In [29]:
ROOT_DIR

'D:\\Fall Dataset\\Dataset CAUCAFall_1\\Dataset CAUCAFall\\CAUCAFALL_AUG'

In [1]:
import sys

In [2]:
sys.path

['/usr/lib/python310.zip',
 '/usr/lib/python3.10',
 '/usr/lib/python3.10/lib-dynload',
 '',
 '/home/ubuntu/additional_drive/venv/lib/python3.10/site-packages']

In [3]:
import sys
sys.path.append('/home/ubuntu/additional_drive/temp_Training/Fall_Detection')
from src.augmentations import run_augmentation
from src.configs import ROOT_DIR
sys.path.remove('/home/ubuntu/additional_drive/temp_Training/Fall_Detection')

>>> Loading src package ...
>>> Training class ...


In [4]:
run_augmentation(ROOT_DIR)

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/244 [00:00<?, ?it/s]

  0%|          | 0/293 [00:00<?, ?it/s]

  0%|          | 0/249 [00:00<?, ?it/s]

  0%|          | 0/213 [00:00<?, ?it/s]

...

  0%|          | 0/235 [00:00<?, ?it/s]

  0%|          | 0/190 [00:00<?, ?it/s]

  0%|          | 0/217 [00:00<?, ?it/s]

  0%|          | 0/217 [00:00<?, ?it/s]

  0%|          | 0/253 [00:00<?, ?it/s]

In [37]:
for person_folder in tqdm(os.listdir(ROOT_DIR)):
    person_folder_path = os.path.join(ROOT_DIR, person_folder)
    
    for activity in os.listdir(person_folder_path):
        activity_folder_path = os.path.join(person_folder_path, activity)
        try:
            
            augment_video_frames(activity_folder_path, activity_folder_path)
            
        except Exception as e:
            print(traceback.format_exc())
            print(e)
        

In [3]:
from src.run import Runner
from src.configs import ROOT_DIR

>>> Loading src package ...
>>> Training class ...


In [4]:
ROOT_DIR

'/home/ubuntu/additional_drive/temp_Training/Dataset_CAUCAFall/CAUCAFall'

In [4]:
args = {
    'data_dir' : ROOT_DIR, 
    'max_frames' : 300,
}

In [5]:
runner = Runner(**args)

2025-09-16 02:36:47,757 - INFO - Dataset Stats:
{'videos': 100, 'frames': 79971, 'frames_per_video': {'Fall backwards': {'Original': 508, 'Total': 1267}, 'Fall forward': {'Original': 300, 'Total': 747}, 'Fall left': {'Original': 418, 'Total': 1042}, 'Fall right': {'Original': 382, 'Total': 952}, 'Fall sitting': {'Original': 436, 'Total': 1087}, 'Hop': {'Original': 428, 'Total': 1067}, 'Kneel': {'Original': 436, 'Total': 1087}, 'Pick up object': {'Original': 444, 'Total': 1107}, 'Sit down': {'Original': 480, 'Total': 1197}, 'Walk': {'Original': 472, 'Total': 1177}}}


In [6]:
runner.__dict__

{'data_dir': 'D:\\Fall Dataset\\Dataset CAUCAFall_1\\Dataset CAUCAFall\\CAUCAFALL_AUG',
 'max_frames': 300,
 'batch_size': 4,
 'lr': 0.0001,
 'epochs': 10,
 'num_classes': 2,
 'train_test_split': [0.85, 0.15],
 'device': 'cuda',
 'transform': Compose(
     Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
     ToTensor()
     Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
 )}

In [14]:
runner.data_dir

'data_dir'

# Actual Training

In [1]:
import sys

In [2]:
sys.path.append('/additional_drive/temp_Training/Fall_Detection')
from src.run import Runner
sys.path.remove('/additional_drive/temp_Training/Fall_Detection')

>>> Loading src package ...
>>> Training class ...


In [5]:
args = {
    'data_dir' : '', 
    'max_frames' : 300,
    'epochs':120
}

In [6]:
video_runner = Runner(**args)

2025-11-29 14:30:02,084 - INFO - Dataset Stats:
{'videos': 100, 'frames': 80004, 'frames_per_video': {'Fall forward': {'Original': 300, 'Total': 747}, 'Pick up object': {'Original': 444, 'Total': 1107}, 'Hop': {'Original': 428, 'Total': 1067}, 'Fall left': {'Original': 418, 'Total': 1042}, 'Sit down': {'Original': 480, 'Total': 1197}, 'Walk': {'Original': 472, 'Total': 1177}, 'Fall right': {'Original': 382, 'Total': 952}, 'Fall sitting': {'Original': 436, 'Total': 1087}, 'Kneel': {'Original': 436, 'Total': 1087}, 'Fall backwards': {'Original': 508, 'Total': 1267}}}


In [ ]:
video_runner.run()

2025-11-29 14:30:03,057 - INFO - Train videos: 85
2025-11-29 14:30:03,057 - INFO - Val videos:   15


🚀 Initializing pipeline...


/home/ubuntu/additional_drive/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ubuntu/additional_drive/venv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
2025-11-29 14:30:03,289 - INFO - 🏋️ Training started...
Iteration:   0%|          | 0/120 [00:00<?, ?it/s]


Epoch [1/120]



Training: 100%|██████████| 22/22 [02:15<00:00,  6.14s/it]

Iteration:   1%|          | 1/120 [02:42<5:21:29, 162.10s/it]

...


Training: 100%|██████████| 22/22 [02:12<00:00,  6.03s/it]

Iteration:  20%|██        | 24/120 [1:04:08<4:17:17, 160.81s/it]

Train Loss: 0.0092 | Train Acc: 1.0000
Val Loss:   0.0215 | Val Acc:   1.0000

Epoch [25/120]



Training: 100%|██████████| 22/22 [02:14<00:00,  6.10s/it]

Iteration:  21%|██        | 25/120 [1:06:49<4:14:23, 160.67s/it]

Train Loss: 0.0341 | Train Acc: 0.9765
Val Loss:   0.0205 | Val Acc:   1.0000

Epoch [26/120]



Training:  59%|█████▉    | 13/22 [01:26<01:01,  6.83s/it]


In [3]:
import os

In [9]:
os.path.exists('/home/ubuntu/additional_drive/temp_Training/Dataset_CAUCAFall/CAUCAFall')

True

In [8]:
os.getcwd()

'/home/ubuntu/additional_drive/temp_Training/Fall_Detection'

In [2]:
from src.model.model import VideoClassificationModel, SimpleAttention

>>> Loading src package ...
>>> Training class ...


In [7]:
model = VideoClassificationModel(hidden_size=256, num_classes=2)

In [3]:
import torchvision

In [5]:
with torch.serialization.safe_globals([VideoClassificationModel, torch.nn.modules.container.Sequential, torch.nn.modules.conv.Conv2d, torch.nn.modules.batchnorm.BatchNorm2d,
                                      torch.nn.modules.activation.ReLU, torch.nn.modules.pooling.MaxPool2d, torchvision.models.resnet.BasicBlock,
                                      torch.nn.modules.pooling.AdaptiveAvgPool2d, torch.nn.modules.rnn.LSTM, SimpleAttention, torch.nn.modules.linear.Linear,
                                      ]):
    model = torch.load('/home/ubuntu/addtitional_drive/temp_Training/Dataset_CAUCAFall/CAUCAFall/weights/full_model.pth', weights_only=True)

In [6]:
model.state_dict()

OrderedDict([('model.0.weight',
              tensor([[[[-1.0419e-02, -6.1356e-03, -1.8098e-03,  ...,  5.6615e-02,
                          1.7083e-02, -1.2694e-02],
                        [ 1.1083e-02,  9.5276e-03, -1.0993e-01,  ..., -2.7124e-01,
                         -1.2907e-01,  3.7424e-03],
                        [-6.9434e-03,  5.9089e-02,  2.9548e-01,  ...,  5.1972e-01,
                          2.5632e-01,  6.3573e-02],
                        ...,
                        [-2.7535e-02,  1.6045e-02,  7.2595e-02,  ..., -3.3285e-01,
                         -4.2058e-01, -2.5781e-01],
                        [ 3.0613e-02,  4.0960e-02,  6.2850e-02,  ...,  4.1384e-01,
                          3.9359e-01,  1.6606e-01],
                        [-1.3736e-02, -3.6746e-03, -2.4084e-02,  ..., -1.5070e-01,
                         -8.2230e-02, -5.7828e-03]],
              
                       [[-1.1397e-02, -2.6619e-02, -3.4641e-02,  ...,  3.2521e-02,
                          6.62